In [33]:
import numpy as np
import matplotlib.pyplot as plt
#import qiskit.quantum_info as qi



#from qiskit import Aer
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import ParityMapper, JordanWignerMapper, BravyiKitaevMapper
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
from qiskit_nature import settings
from qiskit_algorithms.optimizers import  SPSA, COBYLA, L_BFGS_B
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit.circuit.library import TwoLocal
from qiskit_algorithms.minimum_eigensolvers import NumPyMinimumEigensolver, VQE
from qiskit.primitives import Estimator


In [34]:
def reset_intermediate_info():
    global intermediate_info
    intermediate_info.clear()
    print(intermediate_info)
    intermediate_info = {
        'nfev ': [],
        'parameters ': [],
        'energy ': [],
        'stddev ': []
    }

def callback (nfev , parameters , energy , stddev ):
    intermediate_info ['nfev ']. append ( nfev )
    intermediate_info ['parameters ']. append ( parameters )
    intermediate_info ['energy ']. append ( energy )
    intermediate_info ['stddev ']. append ( stddev )

def exact_solver(qubit_op, problem):
    sol = NumPyMinimumEigensolver().compute_minimum_eigenvalue(qubit_op)
    result = problem.interpret(sol)
    return result

def fermion_to_qubit(problem, second_q_op, mapper_name,  z2symmetry_reduction=None):
  if mapper_name == "JW":
    mapper = JordanWignerMapper()
  if mapper_name == "Pa":
    mapper = ParityMapper(num_particles=problem.num_particles)
  if mapper_name == "BK":
    mapper = BravyiKitaevMapper()

  qubit_op = mapper.map(second_q_op)

  if z2symmetry_reduction != None:
    mapper = problem.get_tapered_mapper(mapper)
    qubit_op = mapper.map(second_q_op)
  else :
    qubit_op = mapper.map(second_q_op)

  return qubit_op , mapper

In [35]:
def least_Energy(as_problem):
    core = as_problem.hamiltonian.constants['ActiveSpaceTransformer']
    repulsion = as_problem.hamiltonian.constants['nuclear_repulsion_energy']
    dimer_energy = repulsion + core
    Co_Li_energy = dimer_energy

    return Co_Li_energy

In [36]:
basis = 'sto3g'
settings.use_pauli_sum_op = False

C = 1.9132
L = 2.07
theta = np.deg2rad(94.72)

Co = (0,0,0)
O_1 = (C,0,0)
O_2 = (C*np.cos(theta),C*np.sin(theta),0)
Li = (C+L*np.cos(np.pi-theta),-L*np.sin(np.pi-theta),0)


## 5. Co=O (Double bonding)

In [37]:
Co_O_dimer_atoms = ["Co", "O"]
Co_O_dimer_coords = [Co, O_2]
Co_O_dimer_charge = 0
Co_O_dimer_multiplicity = 2
Co_O_moleculeinfo = MoleculeInfo(Co_O_dimer_atoms, Co_O_dimer_coords, charge=Co_O_dimer_charge, multiplicity=Co_O_dimer_multiplicity)

driver = PySCFDriver.from_molecule(Co_O_moleculeinfo, basis=basis)
E_problem = driver.run()
num_spatial_orbitals = E_problem.num_spatial_orbitals
num_particles = E_problem.num_particles
as_transformer = ActiveSpaceTransformer((6,5), 8)
as_problem = as_transformer.transform(E_problem)
E_5= least_Energy(as_problem)

print(E_5)



-1411.3324180539078


## 6.  Co-O (Single bonding)

In [38]:
Co_O_dimer_atoms = ["Co", "O"]
Co_O_dimer_coords = [Co, O_1]
Co_O_dimer_charge = 0
Co_O_dimer_multiplicity = 4
Co_O_moleculeinfo = MoleculeInfo(Co_O_dimer_atoms, Co_O_dimer_coords, charge=Co_O_dimer_charge, multiplicity=Co_O_dimer_multiplicity)
driver = PySCFDriver.from_molecule(Co_O_moleculeinfo, basis=basis)
E_problem = driver.run()
num_spatial_orbitals = E_problem.num_spatial_orbitals
num_particles = E_problem.num_particles
as_transformer = ActiveSpaceTransformer((7,4), 8)
as_problem = as_transformer.transform(E_problem)
E_6 = least_Energy(as_problem)

print(E_6)

-1411.311118818726


## 1. O-Li(near, bonding)

In [39]:
O_Li_dimer_atoms = ["O", "Li"]
O_Li_dimer_coords = [O_1, Li]
O_Li_dimer_charge = 0
O_Li_dimer_multiplicity = 2

O_Li_moleculeinfo = MoleculeInfo(O_Li_dimer_atoms, O_Li_dimer_coords, charge=O_Li_dimer_charge, multiplicity=O_Li_dimer_multiplicity)
driver = PySCFDriver.from_molecule(O_Li_moleculeinfo, basis=basis)
E_problem = driver.run()
num_spatial_orbitals = E_problem.num_spatial_orbitals
num_particles = E_problem.num_particles
as_transformer = ActiveSpaceTransformer((3,2), 4)
as_problem = as_transformer.transform(E_problem)
E_1 = least_Energy(as_problem)

print(E_1)

-73.24519744420762


## 2. O--Li(far, non-bonding)

In [40]:
O_Li_dimer_atoms = ["O", "Li"]
O_Li_dimer_coords = [O_2, Li]
O_Li_dimer_charge = 0
O_Li_dimer_multiplicity = 4

O_Li_moleculeinfo = MoleculeInfo(O_Li_dimer_atoms, O_Li_dimer_coords, charge=O_Li_dimer_charge, multiplicity=O_Li_dimer_multiplicity)

driver = PySCFDriver.from_molecule(O_Li_moleculeinfo, basis=basis)
E_problem = driver.run()
num_spatial_orbitals = E_problem.num_spatial_orbitals
num_particles = E_problem.num_particles
as_transformer = ActiveSpaceTransformer((4,1), 4)
as_problem = as_transformer.transform(E_problem)
E_2 = least_Energy(as_problem)

print(E_2)

-73.89377989460762


## 3. Co-Li (non-bonding)

In [41]:
Co_Li_dimer_atoms = ["Co", "Li"]
Co_Li_dimer_coords = [Co, Li]
Co_Li_dimer_charge = 0
Co_Li_dimer_multiplicity = 5
energy_arr =[]
order=[]
Co_Li_moleculeinfo = MoleculeInfo(Co_Li_dimer_atoms, Co_Li_dimer_coords, charge=Co_Li_dimer_charge, multiplicity=Co_Li_dimer_multiplicity)

driver = PySCFDriver.from_molecule(Co_Li_moleculeinfo, basis=basis)
E_problem = driver.run()
num_spatial_orbitals = E_problem.num_spatial_orbitals
num_particles = E_problem.num_particles
as_transformer = ActiveSpaceTransformer((6,2), 6)
as_problem = as_transformer.transform(E_problem)
E_3 = least_Energy(as_problem)

print(E_3)

-1357.667308088144


## 4. O-O

In [42]:
O_O_dimer_atoms = ["O", "O"]
O_O_dimer_coords = [O_1, O_2]
O_O_dimer_charge = 0
O_O_dimer_multiplicity = 3
energy_arr =[]
order=[]
O_O_moleculeinfo = MoleculeInfo(O_O_dimer_atoms, O_O_dimer_coords, charge=O_O_dimer_charge, multiplicity=O_O_dimer_multiplicity)

driver = PySCFDriver.from_molecule(O_O_moleculeinfo, basis=basis)
E_problem = driver.run()
num_spatial_orbitals = E_problem.num_spatial_orbitals
num_particles = E_problem.num_particles
as_transformer = ActiveSpaceTransformer((6,2), 6)
as_problem = as_transformer.transform(E_problem)
E_4 = least_Energy(as_problem)

print(E_4)

-131.43760293046435


In [44]:
freeze_arr = [E_1,E_2,E_3,E_4,E_5,E_6]

print(freeze_arr)

[np.float64(-73.24519744420762), np.float64(-73.89377989460762), np.float64(-1357.667308088144), np.float64(-131.43760293046435), np.float64(-1411.3324180539078), np.float64(-1411.311118818726)]


In [52]:
import pickle
import numpy
compound_list_var = ["Co-O", "Co=O", "Co~Li", "Li-O", "Li~O", "O~O"]
compound_list = ["Li-O","Li~O", "Co~Li", "O~O", "Co=O","Co-O"]
ansatz_list = [ "UCCSD","Two local"]
optimizer_list = ["COBYLA", "SPSA", "L-BFGS-B"]

file_dir = "/Users/yoonho/Documents/문서 - 최윤호’s MacBook Air/VQE/FMOVQE/LiCoO2_data_x1/"

for i in range(6):
    compound = compound_list[i]
    for j in range(2):
        ansatz = ansatz_list[j]
        for k in range(3):
            optimizer = optimizer_list[k]

            file_name = f"{compound}/{ansatz}_{optimizer}.pickle"

            with open(file_dir + file_name, 'rb') as f:
                intermediate_data = pickle.load(f)
            
            print(f"min energy of {file_name}: {(min(intermediate_data['energy ']))+freeze_arr[i]}")

min energy of Li-O/UCCSD_COBYLA.pickle: -81.03652971238412
min energy of Li-O/UCCSD_SPSA.pickle: -81.06298214054364
min energy of Li-O/UCCSD_L-BFGS-B.pickle: -81.0697388045102
min energy of Li-O/Two local_COBYLA.pickle: -81.02362627643939
min energy of Li-O/Two local_SPSA.pickle: -80.97841790393854
min energy of Li-O/Two local_L-BFGS-B.pickle: -81.06970269006725
min energy of Li~O/UCCSD_COBYLA.pickle: -81.1175310205899
min energy of Li~O/UCCSD_SPSA.pickle: -81.11795145483565
min energy of Li~O/UCCSD_L-BFGS-B.pickle: -81.11795613080199
min energy of Li~O/Two local_COBYLA.pickle: -81.09423816132369
min energy of Li~O/Two local_SPSA.pickle: -81.03055259412993
min energy of Li~O/Two local_L-BFGS-B.pickle: -81.11757748919891
min energy of Co~Li/UCCSD_COBYLA.pickle: -1373.4297578256983
min energy of Co~Li/UCCSD_SPSA.pickle: -1373.5679455706036
min energy of Co~Li/UCCSD_L-BFGS-B.pickle: -1373.6187392061047
min energy of Co~Li/Two local_COBYLA.pickle: -1373.5763158234156
min energy of Co~Li/Tw